In [ ]:
from flask import Flask, render_template, url_for, request, flash, redirect, jsonify, session
from flask_sqlalchemy import SQLAlchemy
from random import randint
import json
import os
from werkzeug.security import generate_password_hash, check_password_hash
from datetime import datetime


app = Flask(__name__)
app.secret_key = 'KEY'
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///app.db'
db = SQLAlchemy(app)


# Database Models
class blogInfo(db.Model):
    id = db.Column(db.Integer, primary_key = True)
    title = db.Column(db.String(), nullable = False)
    content = db.Column(db.String(200), nullable = False)
    date_uploaded = db.Column(db.DateTime, default = datetime.utcnow)
    
    def __repr__(self):
        return '<Task %r>' % self.id
    
    @property
    def serialize(self):
        return{
            'id': self.id,
            'title': self.title,
            'content': self.content,
            'date_uploaded': self.date_uploaded,
        }
    
class Admin(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(120),nullable=False, unique = True)
    password = db.Column(db.String(120),nullable=False)


# Utility Functions
def retrieve_blogs():
    """
    Return an iterable list of the blogs in the database.
    """
    blogs = blogInfo.query.all()
    serialized_data = []
    for post in blogs:
        serialized_data.append(post.serialize)
    return serialized_data

def initialize_database():
    """
    If a database does not exist, create a new database and populate it with default data.
    """
    if not os.path.exists('app.db'):
        print("No database found. Creating default database now.")
        with app.app_context():
            db.create_all()
        
        with open('sampleposts.json') as sample_posts:
            default_posts = json.loads(sample_posts.read())['blog_posts']
        
        default_admin=Admin(username = 'admin',password = 'password')
        db.session.add(default_admin)
        for post in default_posts:
            new_blog=blogInfo(title=post['title'],content=post['content'])
            db.session.add(new_blog)
        db.session.commit()
    else:
        print("Using existing database.")



# Routing
#-- retrieving blog by a single id
@app.route('/retr_blog/<int:blog_id>', methods=["GET"])
def retr_single_blog(blog_id):
    blog = blogInfo.filter_by(id=blog_id).first()
    serialized_blog = blog.serialize

@app.route('/delete_blog/<int:blog_id>', methods=["POST"])
def delete_blog(blog_id):
    #post = blogInfo.query.filter_by(id=blog_id).first()
    post = blogInfo.query.get_or_404(blog_id)
    title = post.serialize['title']
    try:
        db.session.delete(post)
        db.session.commit()
        flash(f'Blog post "{title}" deleted successfully.')
        return redirect('/admin')
    except:
        flash(f'Blog post could not be deleted.')
        return redirect('/admin')

@app.route('/')
def home_page():
    return render_template('index.html')

@app.route('/articles')
def articles():
    posts = retrieve_blogs()
    return render_template('articles.html', posts_list = posts)

# Admin interface, and adding a blog post to the database
@app.route('/admin', methods=('GET','POST'))
def admin():
    if 'authenticated' not in session or not session['authenticated']:
        return redirect('/login')
    elif request.method == 'POST':
        if 'title' in request.form:
            print(request.form)
            post_title = request.form['title']
            post_content = request.form['content']
            new_blog=blogInfo(title=post_title,content=post_content)
            db.session.add(new_blog)
            db.session.commit()    
            # blog_id = getattr(new_blog, "id")
            flash(f'Blog post "{post_title}" added successfully!')
            return redirect('/admin')
        elif 'logout' in request.form:
            session['authenticated'] = False
            return redirect('/')
    else:
        posts = retrieve_blogs()
        return render_template('admin.html', posts_list = posts)

@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        username = request.form['username']
        password = request.form['password']

        admin = Admin.query.filter_by(username=username).first()

        if admin and admin.password == password:
            session['authenticated'] = True
            return redirect('/admin')
        else:
            flash("Error: Username or password incorrect")
            return render_template('login.html')
    else:
        return render_template('login.html')


    


# Error Code Handling
@app.errorhandler(404)
def not_found(e):
    return render_template('404.html'), 404

def main():
    initialize_database()    
    app.run()



if __name__ == "__main__":
    main()